In [10]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
NUM_CLASSES = 6
CLASSES = ["sadnesss", "joy", "love", "anger", "fear"]
WEIGHTS = torch.tensor([0.00287505, 0.00246512, 0.01015641, 0.00615233, 0.00702346, 0.02318034])

In [12]:
# Load all data
train_data = pd.read_csv("data/training.csv")
test_data = pd.read_csv("data/test.csv")
validation_data = pd.read_csv("data/validation.csv")

# Separate X's and y's from each other
FEATURE_COLUMNS = ["text"]
LABEL_COLUMN = "label"

X_train = train_data[FEATURE_COLUMNS]
Y_train = train_data[LABEL_COLUMN]

X_test = test_data[FEATURE_COLUMNS]
Y_test = test_data[LABEL_COLUMN]

X_val = validation_data[FEATURE_COLUMNS]
Y_val = validation_data[LABEL_COLUMN]

# These are used to run cross validation
X_train_val = pd.concat([X_train, X_val]) 
Y_train_val = pd.concat([Y_train, Y_test])

# These are used to run val and test for Neural Nets
X_val_test = pd.concat([X_val, X_test])
Y_val_test = pd.concat([Y_val, Y_test])

In [13]:
X_train_maxtoks = X_train['text'].str.len().max()
X_test_maxtoks = X_test['text'].str.len().max()
X_val_maxtoks = X_val['text'].str.len().max() 

max_toks = max(X_train_maxtoks, X_test_maxtoks, X_val_maxtoks)
print(max_toks)

300


# Data Loaders

In [80]:
# Pytorch specific constants
from torch.utils.data import TensorDataset, DataLoader
from extractors.chartok import  CharTokenDataset
from utils.neuralnet import NeuralNetwork
from utils.rnn import LSTMNetwork
from utils.transformer import TransformerEncoder
from utils.trainer import training_loop, evaluate
import torch

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 100

# Standard MLP

In [74]:
# Because an MLP operates on fixed size inputs, we will use the entire fixed size input for this
train_data = CharTokenDataset(X_train, Y_train, tokenizer=None, max_seq_length=max_toks)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

val_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

# Batch size is hard coded to 1 for accuracy purposes. 
test_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks)
test_loader = DataLoader(val_data, batch_size=1, shuffle=True)

In [75]:
mlp = NeuralNetwork(max_toks, [100, 100, 100], NUM_CLASSES, device="cuda")
training_loop(mlp, train_loader, val_loader, epochs=EPOCHS, learning_rate=LEARNING_RATE, weights= WEIGHTS)
torch.save(mlp.state_dict(), "models/mlpseq")

Epoch 1
train_loss = 1.6551, val_loss = 1.6072
Epoch 2
train_loss = 1.7023, val_loss = 1.5840
Epoch 3
train_loss = 1.6816, val_loss = 1.5787
Epoch 4
train_loss = 1.5226, val_loss = 1.5770
Epoch 5
train_loss = 1.5212, val_loss = 1.5732
Epoch 6
train_loss = 1.7088, val_loss = 1.5734
Epoch 7
train_loss = 1.3962, val_loss = 1.5871
Epoch 8
train_loss = 1.4262, val_loss = 1.5724
Epoch 9
train_loss = 1.5318, val_loss = 1.5747
Epoch 10
train_loss = 1.5580, val_loss = 1.5735
Epoch 11
train_loss = 1.6167, val_loss = 1.5733
Epoch 12
train_loss = 1.5476, val_loss = 1.5724
Epoch 13
train_loss = 1.5315, val_loss = 1.5740
Epoch 14
train_loss = 1.6457, val_loss = 1.5734
Epoch 15
train_loss = 1.6191, val_loss = 1.5732
Epoch 16
train_loss = 1.7717, val_loss = 1.5720
Epoch 17
train_loss = 1.7447, val_loss = 1.5715
Epoch 18
train_loss = 1.4541, val_loss = 1.5753
Epoch 19
train_loss = 1.5284, val_loss = 1.5730
Epoch 20
train_loss = 1.5610, val_loss = 1.5746
Epoch 21
train_loss = 1.3930, val_loss = 1.5852
E

In [76]:
evaluate(model=mlp, val_dl=test_loader)

loss = 1.6209
accuracy = 0.3458
f1 = 0.2596


(tensor(1.6209), 0.34575, 0.25962029233755085)

# RNN

In [77]:
train_data = CharTokenDataset(X_train, Y_train, tokenizer=None, max_seq_length=max_toks, dtype=torch.int32)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

val_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks, dtype = torch.int32)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

# Batch size is hard coded to 1 for accuracy purposes. 
test_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks, dtype=torch.int32)
test_loader = DataLoader(val_data, batch_size=1, shuffle=True)


In [78]:
lstm = LSTMNetwork(32,  20, 10, NUM_CLASSES)
training_loop(lstm, train_loader, val_loader, epochs=EPOCHS, learning_rate=LEARNING_RATE, weights = WEIGHTS)
torch.save(lstm.state_dict(), "models/lstmseq")

Epoch 1
train_loss = 1.4301, val_loss = 1.5713
Epoch 2
train_loss = 1.4963, val_loss = 1.5724
Epoch 3
train_loss = 1.6407, val_loss = 1.5704
Epoch 4
train_loss = 1.5011, val_loss = 1.5718
Epoch 5
train_loss = 1.7186, val_loss = 1.5694
Epoch 6
train_loss = 1.8273, val_loss = 1.5701
Epoch 7
train_loss = 1.6810, val_loss = 1.5712
Epoch 8
train_loss = 1.5338, val_loss = 1.5707
Epoch 9
train_loss = 1.6634, val_loss = 1.5712
Epoch 10
train_loss = 1.8470, val_loss = 1.5702
Epoch 11
train_loss = 1.5519, val_loss = 1.5704
Epoch 12
train_loss = 1.4881, val_loss = 1.5711
Epoch 13
train_loss = 1.8763, val_loss = 1.5697
Epoch 14
train_loss = 1.5481, val_loss = 1.5695
Epoch 15
train_loss = 1.5107, val_loss = 1.5700
Epoch 16
train_loss = 1.5635, val_loss = 1.5716
Epoch 17
train_loss = 1.5457, val_loss = 1.5702
Epoch 18
train_loss = 1.6173, val_loss = 1.5705
Epoch 19
train_loss = 1.7106, val_loss = 1.5705
Epoch 20
train_loss = 1.5436, val_loss = 1.5704
Epoch 21
train_loss = 1.6580, val_loss = 1.5702
E

In [79]:
evaluate(model=lstm, val_dl=test_loader)

loss = 1.5700
accuracy = 0.3498
f1 = 0.1813


(tensor(1.5700), 0.34975, 0.18125588071865162)

# Transformer

In [99]:
train_data = CharTokenDataset(X_train, Y_train, tokenizer=None, max_seq_length=max_toks, dtype=torch.int32)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

val_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks, dtype = torch.int32)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

# Batch size is hard coded to 1 for accuracy purposes. 
test_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks, dtype=torch.int32)
test_loader = DataLoader(val_data, batch_size=1, shuffle=True)


In [102]:
transformer = TransformerEncoder(max_toks, NUM_CLASSES, 10, 5, 4, ff=10, dropout=0.1)
training_loop(transformer, train_loader, val_loader, epochs=EPOCHS, learning_rate=LEARNING_RATE, weights = WEIGHTS)
torch.save(transformer.state_dict(), "models/xformerseq")

NameError: name 'WEIGHTS' is not defined

In [83]:
evaluate(model=transformer, val_dl=test_loader)

loss = 1.5704
accuracy = 0.3498
f1 = 0.1813


(tensor(1.5704), 0.34975, 0.18125588071865162)